In [ ]:
import os
import pydicom
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, widgets, Layout, HBox, VBox, Label, Button
from IPython.display import display, clear_output

In [ ]:
path_to_patient_dicomdir = "../../data/ProstateData/BREST patients/BREST_001/DICOMDIR"
dicomdir = pydicom.dcmread(path_to_patient_dicomdir)

In [ ]:
# Go through each patient (there should be only one)
images_to_slide_over = []
series_index_to_load = 0 # set here the series idx that you want to visualize (0 to 6 for patient 001)
for patient in dicomdir.patient_records:

    for study in patient.children:
        for series in study.children:
            desc = series.get('SeriesDescription')
            print("desc:", desc)

            #if desc == "Pelvis_t2_spc_rst_tra_p2_iso":
            if desc == "Pelvis_t2_haste_fs_db_tra_p2_320":
                image_records = [img for img in series.children if img.DirectoryRecordType == "IMAGE"]
                
                for idx, image_record in enumerate(image_records):
                    # Get the path to the image file using the reference in the DICOMDIR
                    img_path = os.path.join(os.path.dirname(path_to_patient_dicomdir), os.path.join(*image_record.ReferencedFileID))
                    
                    # Read and display the image
                    ds = pydicom.dcmread(img_path)

                    if idx == 0:
                        for elem in ds:
                            print(elem)

                    images_to_slide_over.append(ds.pixel_array)
                    # plt.imshow(ds.pixel_array, cmap=plt.cm.bone)
                    # plt.show()
                
                """
                for idx, image_record in enumerate(image_records):
                    print("idx:", idx)
                    # Get the path to the image file using the reference in the DICOMDIR
                    img_path = os.path.join(os.path.dirname('../../data/ProstateData/BREST patients/BREST_001/DICOMDIR'), os.path.join(*image_record.ReferencedFileID))
                    
                    # Read and display the image
                    ds = pydicom.dcmread(img_path)
                    plt.imshow(ds.pixel_array, cmap=plt.cm.bone)
                    plt.show()

                    #for elem in ds:
                    #    print(elem)
                """


In [ ]:
 # Function to plot an image from a list at a specified index
def plot_image(list_of_images, image_idx):
    plt.imshow(list_of_images[image_idx])
    plt.axis('off')  # Turn off axis numbers
    plt.show()

# Function to get a slider widget
def get_slider(max_value):
    return widgets.IntSlider(
        value=0,
        min=0,
        max=max_value,
        step=1,
        description='Image Index:',
        continuous_update=False
    )

# Wrapper function to create an interactive plot with a list of images
def create_interactive_plot(list_of_images):
    slider = get_slider(len(list_of_images) - 1)
    interactive_plot = interactive(plot_image, list_of_images=widgets.fixed(list_of_images), image_idx=slider)
    display(interactive_plot)

In [ ]:
create_interactive_plot(images_to_slide_over)

# Search for segmentation masks

In [ ]:
for record in dicomdir.DirectoryRecordSequence:
    # Check if the record is for an image and has the correct SOP Class UID for segmentation
    if record.DirectoryRecordType == "IMAGE" and record.get("ReferencedSOPClassUIDInFile", "") == "1.2.840.10008.5.1.4.1.1.66.4":
        segmentation_info = {
            "Patient ID": record.get("PatientID", "N/A"),
            "Study ID": record.get("StudyID", "N/A"),
            "Series Number": record.get("SeriesNumber", "N/A"),
            "Instance Number": record.get("InstanceNumber", "N/A"),
            "Referenced File ID": record.get("ReferencedFileID", "N/A")
        }
        print("seg:", segmentation_info)

In [ ]:
for record in dicomdir.DirectoryRecordSequence:
    # Check if the record corresponds to a segmentation object
    if record.get("ReferencedSOPClassUIDInFile", "") == "1.2.840.10008.5.1.4.1.1.66.4":
        has_segmentation_type = "SegmentationType" in record
        has_segment_sequence = "SegmentSequence" in record
        has_referenced_series_sequence = "ReferencedSeriesSequence" in record

        if has_segmentation_type or has_segment_sequence or has_referenced_series_sequence:
            segmentation_info_found = True
            print(f"Segmentation information found in record: {record}")
            if has_segmentation_type:
                print(f"  Segmentation Type: {record.SegmentationType}")
            if has_segment_sequence:
                print(f"  Segment Sequence: {record.SegmentSequence}")
            if has_referenced_series_sequence:
                print(f"  Referenced Series Sequence: {record.ReferencedSeriesSequence}")
            print("\n")

# Analysis of all patients

In [ ]:

all_patients_dir = '../../data/ProstateData/BREST patients/'
study_counts = {}
study_description_counts = {}
i=0

record_df = pd.DataFrame(columns=['patient', 'study', 'series', 'series_modality', 'series_description', 'series_body_part', 'records', 'img_rows', 'img_columns'])

for patient_folder in os.listdir(all_patients_dir):
    patient_dir = all_patients_dir + patient_folder + "/DICOMDIR"
    dcm = pydicom.dcmread(patient_dir)

    # Iterate over studies
    for study in dcm.patient_records[0].children:
        study_count = study_counts.get(patient_folder, 0)
        study_counts[patient_folder] = study_count + 1

        desc = study.get("StudyDescription", "N/A")
        desc_count = study_description_counts.get(desc, 0)
        study_description_counts[desc] = desc_count + 1

        # iterate over series of study
        for series in study.children:
            # count records of the series
            img_records = len([img for img in series.children if img.DirectoryRecordType == "IMAGE"])
            img_rows = -1
            img_columns = -1
            for img in series.children:
                if img.DirectoryRecordType == "IMAGE":
                    img_rows = img.get("Rows")
                    img_columns = img.get("Columns")
                    break
            new_row = [patient_folder, study.get("StudyID"), series.get("SeriesNumber"), series.get('Modality'), series.get('SeriesDescription'), series.get('BodyPartExamined'), img_records, img_rows, img_columns]
            record_df.loc[i] = new_row
            i+=1

In [ ]:
record_df

Analyze patients for which 2 studies were conducted

In [ ]:
pd.set_option('display.max_rows', 200)
patients_2_studies = record_df.groupby(["patient"])["study"].nunique().reset_index(name='counts')
patients_2_studies[patients_2_studies["counts"] != 1]

In [ ]:
# patients with 2 studies are the patients for which a CT scan (and additional PET scan) was taken
record_df[record_df["patient"] == "BREST_132"]

In [ ]:
pd.set_option('display.max_rows', 5)
record_df[record_df['series_body_part']!="ABDOMEN"]

Get number of patients with a given count of CT/MR/PT series taken

In [ ]:
series_modalities = record_df.groupby(["patient", "series_modality"]).size().reset_index()
series_modalities = series_modalities.rename(columns={0: "series_for_patient_count"})

modality_counts = series_modalities.groupby(["series_modality", "series_for_patient_count"]).size()
modality_counts

Get for how many patients each series type was conducted

In [ ]:
record_df_grouped = record_df.groupby(["series_description", "series_modality"]).agg({"patient": "nunique", "records": {'min', 'mean', 'max'}, "img_rows": {'min', 'mean', 'max'}, 'img_columns': {'min', 'mean', 'max'}})
record_df_grouped.sort_values(by=["series_modality", ("patient", "nunique")], ascending=[True, False])

In [ ]:
print("number of studies per patient:", set(study_counts.values()))
print(len([patient for patient in study_counts if study_counts[patient] == 1]))
print(len([patient for patient in study_counts if study_counts[patient] == 2]))

In [ ]:
print("study descriptions with frequencies:", study_description_counts)

In [ ]:
print("Minimum number of images per study type")
print(record_df.groupby(["series_modality"])["records"].min())
print("-"*20)
print("Maximum number of images per study type")
print(record_df.groupby(["series_modality"])["records"].max())

Plot intensity distribution of an image

In [ ]:
dicom_file_path = "../../data/ProstateData/BREST patients/BREST_136/DICOM/00001975/AA0839C2/AAAAF44B/0000D8AB/EE7CC5A3"

dataset = pydicom.dcmread(dicom_file_path)

# Check if the Pixel Data attribute is present in the dataset.
if "PixelData" in dataset:
    pixel_data = dataset.pixel_array
    pixel_data = pixel_data.flatten()
    plt.hist(pixel_data.ravel(), bins=256, range=(0, 255))
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.show()

    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    plt.show()

# Visualizing series per description to study patterns and anomalies

In [ ]:
class SeriesImageNavigator:
    def __init__(self, all_series_images):
        self.all_series_images = all_series_images
        self.current_patient_index = 0
        self.interactive_plot = None
        self.setup_ui()

    def plot_image(self, list_of_images, image_idx):
        if list_of_images and image_idx < len(list_of_images):
            plt.imshow(list_of_images[image_idx])
            plt.axis('off')
            plt.show()
        else:
            print("No images to display for this patient or index out of range.")

    def get_slider(self, max_value):
        return widgets.IntSlider(value=0, min=0, max=max_value, step=1, description='Image Index:', continuous_update=False)

    def update_plot(self, patient_idx):
        self.current_patient_index = patient_idx
        if self.all_series_images[patient_idx]:
            slider = self.get_slider(len(self.all_series_images[patient_idx]) - 1)
            self.interactive_plot = interactive(self.plot_image, list_of_images=widgets.fixed(self.all_series_images[patient_idx]), image_idx=slider)
            self.display_container.children = [self.nav_buttons, self.interactive_plot]
        else:
            print(f"No images found for patient at index {patient_idx}")

    def navigate_patient(self, b):
        if b.description == 'Next' and self.current_patient_index < len(self.all_series_images) - 1:
            self.update_plot(self.current_patient_index + 1)
        elif b.description == 'Previous' and self.current_patient_index > 0:
            self.update_plot(self.current_patient_index - 1)

    def setup_ui(self):
        self.interactive_plot = interactive(self.plot_image, list_of_images=widgets.fixed(self.all_series_images[self.current_patient_index]), image_idx=self.get_slider(len(self.all_series_images[self.current_patient_index]) - 1))

        next_button = Button(description='Next', layout=Layout(width='100px'))
        prev_button = Button(description='Previous', layout=Layout(width='100px'))
        next_button.on_click(self.navigate_patient)
        prev_button.on_click(self.navigate_patient)

        self.nav_buttons = HBox([prev_button, next_button])
        self.display_container = widgets.VBox([self.nav_buttons, self.interactive_plot])

    def display(self):
        display(self.display_container)

In [ ]:
# Collect all images for each patient
def collect_images_of_series(series_description: str):
    all_series_images = []
    all_patients_dir = '../../data/ProstateData/BREST patients/'
    for patient_folder in os.listdir(all_patients_dir):
        patient_dir = os.path.join(all_patients_dir, patient_folder, "DICOMDIR")
        dcm = pydicom.dcmread(patient_dir)
        for study in dcm.patient_records[0].children:
            for series in study.children:
                if str(series.SeriesDescription).lower().strip() == series_description.lower().strip():
                    image_records = [img for img in series.children if img.DirectoryRecordType == "IMAGE"]
                    image_series = []
                    for image_record in image_records:
                        # Get the path to the image file using the reference in the DICOMDIR
                        img_path = os.path.join(os.path.dirname(patient_dir), os.path.join(*image_record.ReferencedFileID))
                        
                        # Read and display the image
                        ds = pydicom.dcmread(img_path)
                        image_series.append(ds.pixel_array)
                    all_series_images.append(image_series)

    return all_series_images

In [ ]:
# Collect numpy arrays of images pertaining to a series with a given description
directory_path = '../../data/ProstateData/BREST patients/'
series_images = collect_images_of_series("Pelvis_t1_tse_cor_p2")


In [ ]:
navigator = SeriesImageNavigator(series_images)
navigator.display()

### Visualize series of the same patient in parallel

In [ ]:
class PatientImageNavigator:
    def __init__(self, series):
        self.series_images = series
        self.interactive_plots = []
        self.setup_ui()

    def plot_image(self, list_of_images, image_idx):
        if list_of_images and image_idx < len(list_of_images):
            plt.imshow(list_of_images[image_idx])
            plt.axis('off')
            plt.show()
        else:
            print("No images to display or index out of range.")

    def get_slider(self, max_value):
        return widgets.IntSlider(value=0, min=0, max=max_value, step=1, description='Image Index:', continuous_update=False)

    def setup_ui(self):
        row_plots = []
        for i, series in enumerate(self.series_images):
            if series['images']:
                title = series['title'] 
                slider= self.get_slider(len(series['images'][0]) - 1)
                interactive_plot = interactive(self.plot_image, list_of_images=widgets.fixed(series['images'][0]), image_idx=slider)
                plot_container = widgets.VBox([Label(title), interactive_plot], layout=Layout(border='1px solid black', padding='5px'))
                row_plots.append(plot_container)

                if len(row_plots) == 4:
                    self.interactive_plots.append(HBox(row_plots, layout=Layout(padding='5px')))
                    row_plots = []
        
        if row_plots:
            self.interactive_plots.append(HBox(row_plots, layout=Layout(padding='5px')))

        self.display_container = VBox(self.interactive_plots, layout=Layout(padding='5px'))

    def display(self):
        display(self.display_container)


In [ ]:
def create_input_for_navigator(dataset_path, patient_folder, descriptions):
    res = []
    for description in descriptions:
        try:
            series_npy = np.load(os.path.join(dataset_path, patient_folder, description + '.npy'))
            res.append({
                'title': description,
                'images': [[series_npy[i, :, :] for i in range(series_npy.shape[0])]] #convert to a list of 2D arrays
            })
        except Exception as e:
            continue # series does not exist
    return res

In [ ]:
descriptions = [
    'Pelvis_t2_haste_fs_db_tra_p2_320',
    '*MRAC_PET_mlaa_siemens_4BP TK_AC Images',
    '*Pelvis_MRAC_PET_mlaa_siemens_Becken_1BP_15min_LM_AC Images',
    'Pelvis_ep2d_diff_tra_ADC',
    'Pelvis_ep2d_diff_tra',
    'Pelvis_t1_tse_cor_p2',
    'Pelvis_t2_spc_rst_tra_p2_iso'
]
input = create_input_for_navigator('../../data/ProstateDataCenterCropNP','BREST_137', descriptions)

In [ ]:
navigator = PatientImageNavigator(input)
navigator.display()